In [1]:
!pip install transformers
!pip install sentencepiece
# !pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json
import torch
import pandas as pd
# import swifter

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
qa_tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")
qa_model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer").to(device)

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [4]:
prop_tokenizer = AutoTokenizer.from_pretrained("chentong00/propositionizer-wiki-flan-t5-large")
prop_model = AutoModelForSeq2SeqLM.from_pretrained("chentong00/propositionizer-wiki-flan-t5-large").to(device)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [5]:
def generate_qa_pairs(context, num_pairs):
  inputs = qa_tokenizer(context, return_tensors="pt")
  inputs.to(device)
  qa_pairs = []

  for i in range(num_pairs):
    outputs = qa_model.generate(**inputs, max_length=100, do_sample=True)
    question_answer = qa_tokenizer.decode(outputs[0], skip_special_tokens=False)
    question_answer = question_answer.replace(qa_tokenizer.pad_token, "").replace(qa_tokenizer.eos_token, "")
    try:
      question, answer = question_answer.split(qa_tokenizer.sep_token)

      qa_pair = {"question": question.lstrip(' ').rstrip(' '), "answer": answer.lstrip(' ').rstrip(' ') + "."}
      qa_pairs.append(qa_pair)
    except:
      print("FAILED TO SEPERATE QA")
      print(question_answer)
      print(context)

  return qa_pairs

In [6]:
def extract_propositions(sentence):
    title = ""
    section = ""
    content = sentence

    input_text = f"Title: {title}. Section: {section}. Content: {content}"

    input_ids = prop_tokenizer(input_text, return_tensors="pt").input_ids
    input_ids.to(device)
    outputs = prop_model.generate(input_ids.to(device), max_new_tokens=512).cpu()

    output_text = prop_tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        prop_list = json.loads(output_text)
    except:
        prop_list = []
        print("[ERROR] Failed to parse output text as JSON.")
        print(output_text)
        return prop_list

    return prop_list

In [7]:
# generate_qa_pairs("In Khatlon, 67% of people have access to improved water.", 5)

In [8]:
# generate_qa_pairs("Young women with low empowerment would like 2 children more than young women with low empowerment.", 10)

In [9]:
# extract_propositions("The Shelby Mustang is a high performance variant of the Ford Mustang which was built by Shelby American from 1965 to 1968, and from 1969 to 1970 by Ford.")

In [10]:
# # BEST ONE
# import nltk
# from nltk.tokenize import word_tokenize

# nltk.download('punkt')

# def get_best_overlap_qa_pair(proposition, qa_pairs):
#     proposition_tokens = set(word_tokenize(proposition.lower()))

#     best_overlap = 0
#     best_qa_pair = None

#     for question, answer in qa_pairs:
#         answer_tokens = set(word_tokenize(answer.lower()))
#         overlap = len(answer_tokens & proposition_tokens)

#         if overlap > best_overlap:
#             best_overlap = overlap
#             best_qa_pair = (question, answer)

#     return best_qa_pair

# def select_qa_pairs(proposition, qa_pairs):
#     best_qa_pair = get_best_overlap_qa_pair(proposition, qa_pairs)

#     if best_qa_pair is not None:
#         return [best_qa_pair]
#     else:
#         return []

# propositions = [
#     "The Shelby Mustang is a high-performance variant of the Ford Mustang.",
#     "The Shelby Mustang was built by Shelby American from 1965 to 1968.",
#     "The Shelby Mustang was built by Ford from 1969 to 1970."
# ]

# qa_pairs = [
#     ("During what years was the Shelby Mustang built?", "1965 to 1968"),
#     ("By the time the Shelby was built how did Ford introduce the Shelby Mustang?", "1969 to 1970"),
#     ("What company built the Shelby Mustang from 1965 to 1968?", "Shelby American"),
#     ("Which company built the Shelby Mustang?", "Shelby American"),
#     ("Who built the Shelby Mustang?", "Shelby American"),
#     ("What company built the Shelby Mustang?", "Shelby American"),
#     ("What is a Shelby Mustang?", "high-performance variant of the Ford Mustang"),
#     ("Why was the Shelby Mustang created?", "high-performance variant"),
#     ("When was Shelby Mustang built by Shelby American?", "1965 to 1968"),
#     ("What company built the Shelby Mustang?", "Shelby American")
# ]

# result = []

# for proposition in propositions:
#     selected_qa_pairs = select_qa_pairs(proposition, qa_pairs)
#     result.append({
#         'proposition': proposition,
#         'selected_qa_pairs': selected_qa_pairs
#     })

# # Print the result
# for r in result:
#     print("Proposition:", r['proposition'])
#     print("Selected QA Pair:", r['selected_qa_pairs'])
#     print("\n")


In [11]:
# def select_qa_pair(question_answer_pairs, proposition):
#   proposition = proposition.rstrip('.')

#   filtered_pair = question_answer_pairs[0]
#   max_length = 0

#   for pair in question_answer_pairs:
#       question, answer = pair["question"], pair["answer"]

#       if proposition.endswith(answer) and len(answer) >= max_length:
#           filtered_pair = pair
#           max_length = len(answer)

#   return filtered_pair

def rightmost_qa_pairs(question_answer_pairs, proposition):
  proposition = proposition.rstrip('.')

  filtered_pairs = []

  filtered_pair = question_answer_pairs[0]
  max_length = 0

  for pair in question_answer_pairs:
      question, answer = pair["question"], pair["answer"]

      if proposition.endswith(answer):
          filtered_pairs.append(pair)

  return filtered_pairs if len(filtered_pairs) != 0 else question_answer_pairs

  # filtered_questions = filter_questions(proposition, question_answer_pairs)
  # print(filtered_questions)

In [12]:
def most_similar_pair(pairs, sentence):
  # question_list = [entry["question"] for entry in sentence_list]

  # Tokenize input sentences
  tokenized_input = set(sentence.lower().split())

  # Initialize variables to track maximum overlap and corresponding sentence
  max_overlap = 0
  most_overlapping_sentence = {}

  for pair in pairs:
    candidate_sentence = pair['question']
    # Tokenize candidate sentence
    tokenized_candidate = set(candidate_sentence.lower().split())

    # Calculate word overlap
    overlap_count = len(tokenized_input.intersection(tokenized_candidate))

    # Update maximum overlap and corresponding sentence if needed
    if overlap_count > max_overlap:
        max_overlap = overlap_count
        most_overlapping_sentence = pair

  return most_overlapping_sentence

In [13]:
def concat_qa_pairs(qa_pairs):
  try:
    joined_qa_pairs = " ".join(f"{qa_pair['answer']} {qa_pair['question']} |" for qa_pair in qa_pairs)
    # return "Blueprint: " + joined_qa_pairs + " Summary: "
    return joined_qa_pairs
  except:
    print("no blueprint could be made")

In [14]:
def pipeline(reference):
  # print("Reference:")
  # print(reference)
  # print()
  propositions = extract_propositions(reference)

  all_qa_pairs = []

  for proposition in propositions:
    # print(proposition)
    # print()
    qa_pairs = generate_qa_pairs(proposition, 5)
    # print(qa_pairs)
    # print()
    # right_pairs = rightmost_qa_pairs(qa_pairs, proposition)
    pair = most_similar_pair(qa_pairs, proposition)
    # print(pair)
    # print()
    all_qa_pairs.append(pair)

  return concat_qa_pairs(all_qa_pairs)

In [15]:
# pipeline("Access to improved water varies by region, from 67% Khatlon to 99% in Dushanbe.")

In [16]:
# pipeline("Overall, 97% of households in Tajikistan have access to an improved and not shared toilet facility.")

In [17]:
# pipeline("Women in GBAO and Dushanbe are most likely to have gone to higher education (28% and 27%, respectively) while only 4% of women in DRS and 5% of women in Khatlon have attended higher education.")

In [18]:
# pipeline("In Nigeria, young women with high empowerment are much more likely to be currently using contraception than those with low empowerment, and among non-users, those with high empowerment are much more likely to intend to use in the future than those with low empowerment.")

In [19]:
import ast

# df = pd.read_csv('train_skip_no_refs_eng_blueprints.csv', delimiter="\t")
df = pd.read_csv('dev.csv', delimiter="\t")
# df['table_text'] = df['table_text'].apply(ast.literal_eval)

In [20]:
len(df)

754

In [21]:
# df.iloc[4153]

IndexError: ignored

In [22]:
def process_train_dataframe(df):
    df['blueprint'] = ''
    print(len(df))
    print()

    for index, row in df.iterrows():
        print(index)

        example_id = row['example_id']
        table_text = row['table_text']

        # print(table_text)

        if '-en-' in example_id:
            df.at[index, 'blueprint'] = []  # Use df.at to modify the original DataFrame

            for i in range(0, len(row['table_text'])):
                # print("Table text:")
                # print(row['table_text'])
                # print()
                blueprint = pipeline(row['table_text'][i])
                df.at[index, 'blueprint'].append(blueprint)

    return df

In [23]:
def process_dev_dataframe(df):
    df['blueprint'] = ''
    print(len(df))
    print()

    for index, row in df.iterrows():
        print(index)

        example_id = row['example_id']
        table_text = row['target']

        # print(table_text)

        if '-en-' in example_id:
            df.at[index, 'blueprint'] = []  # Use df.at to modify the original DataFrame

            blueprint = pipeline(table_text)
            df.at[index, 'blueprint'] = blueprint

    return df

In [24]:
df = process_dev_dataframe(df)

754

0
1
FAILED TO SEPERATE QA
 There is significant diversity in the family formation process across countries
There is significant diversity in the family formation process across countries.
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
FAILED TO SEPERATE QA
 Nairobi's foetility rank is the lowest in the province
Fertility is lowest in Nairobi province.
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
FAILED TO SEPERATE QA
 Figure 9.1 is intended to assess changes in coverage indicators.
The purpose of Figure 9.1 is to assess the magnitude of recent changes in key coverage indicators.
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
FAILED TO SEPERATE QA
 Seve

In [ ]:
# df.iloc[4153]

In [25]:
df.to_csv('dev_eng_blueprints_only_eng.csv', sep='\t', index=False)

In [ ]:
# import pandas as pd

# # Read CSV file with delimiter '\t' (tab-separated)
# df = pd.read_csv('train_skip_no_refs_eng_blueprints_only_eng.csv', delimiter='\t')

# # Find rows where "-en-" is in example_id and blueprint is NaN
# condition = (df['example_id'].str.contains('-en-')) & pd.isna(df['blueprint'])
# result_df = df[condition]

# # Print or manipulate the resulting DataFrame
# print(result_df)